In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving MahaSent_ST_Test.csv to MahaSent_ST_Test (1).csv
Saving MahaSent_ST_Train.csv to MahaSent_ST_Train (1).csv
Saving MahaSent_ST_Val.csv to MahaSent_ST_Val (1).csv
Saving tweets-test_PT.csv to tweets-test_PT.csv
Saving tweets-train_PT.csv to tweets-train_PT (1).csv
Saving tweets-valid_PT.csv to tweets-valid_PT (1).csv


In [ ]:
pt_train_df = pd.read_csv("tweets-train_PT.csv")
pt_val_df = pd.read_csv("tweets-valid_PT.csv")
pt_test_df = pd.read_csv('tweets-test_PT.csv')
st_train_df = pd.read_csv("MahaSent_ST_Train.csv")
st_val_df = pd.read_csv("MahaSent_ST_Val.csv")
st_test_df = pd.read_csv('MahaSent_ST_Test.csv')

In [ ]:
print(pt_train_df.head())
print("\n")
print(st_train_df.head())

                                               tweet  label
0  ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...     -1
1  सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...     -1
2  उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...     -1
3  आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...      1
4  बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...     -1


   Unnamed: 0                                  marathi_text  label
0           0                              मी तो मूर्ख आहे.     -1
1           1  आईने आत्महत्या केली आणि माझे वडील पळून गेले.     -1
2           2                   - त्यांनी फक्त जॉयला मारले?     -1
3           3            मला माफ करा मी आज तिथेच केले नाही.     -1
4           4                            नाही, नक्कीच नाही.     -1


In [ ]:
# Standardize column names by renaming text fields to 'text'

#Training
pt_train_df = pt_train_df.rename(columns={"tweet": "text"})
st_train_df = st_train_df.rename(columns={"marathi_text": "text"})
pt_train_df = pt_train_df[["text", "label"]]
st_train_df = st_train_df[["text", "label"]]

# Combine MR and PT datasets into a single train dataframe
train_df = pd.concat([pt_train_df, st_train_df], ignore_index=True)

# Shuffle the datasets to avoid ordering bias during training
train_df = train_df.sample(frac=1, random_state=42)

In [ ]:
#similarly for validation and test
# Validation
pt_val_df = pt_val_df.rename(columns={"tweet": "text"})
st_val_df = st_val_df.rename(columns={"marathi_text": "text"})
val_df = pd.concat([pt_val_df[["text", "label"]], st_val_df[["text", "label"]]], ignore_index=True)
val_df = val_df.sample(frac=1, random_state=42)
# Test
pt_test_df = pt_test_df.rename(columns={"tweet": "text"})
st_test_df = st_test_df.rename(columns={"marathi_text": "text"})
test_df = pd.concat([pt_test_df[["text", "label"]], st_test_df[["text", "label"]]], ignore_index=True)
test_df = test_df.sample(frac=1, random_state=42)

In [ ]:
# Convert original sentiment labels to numeric IDs for model training
# Also ensure 'text' column contains only strings and handle NaN values

for df in [train_df, val_df, test_df]:
    df["label"] = df["label"].astype(int)
    df["label"] = df["label"] + 1  # -1->0, 0->1, 1->2
    df["text"] = df["text"].astype(str).fillna("") # Ensure text is string and handle NaNs

In [ ]:
# Convert pandas dataframes into HuggingFace Dataset objects

train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

In [ ]:
# Tokenize text: pad, truncate, and encode inputs for BERT

def tokenize_fn(batch):
    # Ensure each text item is explicitly a string within a list for tokenizer compatibility
    texts = [str(x) for x in batch["text"]]
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

In [ ]:
# Load Marathi-BERT-v2 tokenizer and model for sequence classification

model_name = "l3cube-pune/marathi-bert-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: pos/neg/neutral

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encoded = train_dataset.map(tokenize_fn, batched=True)
val_encoded   = val_dataset.map(tokenize_fn, batched=True)
test_encoded  = test_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/24114 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3750 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define accuracy, precision, recall, and F1 score metrics

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

In [ ]:
# Configure training parameters such as epochs, batch size, and learning rate

training_args = TrainingArguments(
    output_dir="./mr_gt_marathi_bert_v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    fp16=True,             # if your GPU supports it
    report_to="none"
)


In [ ]:
# Initialize Trainer and begin fine-tuning Marathi-BERT-v2

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-3285206523.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,0.509600,0.442510,0.828333,0.834776,0.828333,0.829989
2,0.467500,0.416194,0.845000,0.845252,0.845000,0.844340
3,0.312800,0.444061,0.845000,0.844287,0.845000,0.844446


TrainOutput(global_step=4524, training_loss=0.4503846400401742, metrics={'train_runtime': 1110.947, 'train_samples_per_second': 65.117, 'train_steps_per_second': 4.072, 'total_flos': 4758537716319744.0, 'train_loss': 0.4503846400401742, 'epoch': 3.0})

In [ ]:
# Evaluate the fine-tuned model on the combined test set

test_metrics = trainer.evaluate(test_encoded)
print("Test metrics:", test_metrics)


Test metrics: {'eval_loss': 0.4308089315891266, 'eval_accuracy': 0.8344, 'eval_precision_macro': 0.8347314846548698, 'eval_recall_macro': 0.8344, 'eval_f1_macro': 0.8336700953365819, 'eval_runtime': 10.1857, 'eval_samples_per_second': 368.164, 'eval_steps_per_second': 23.072, 'epoch': 3.0}


In [ ]:
pt_test_df_processed = pt_test_df.copy()
pt_test_df_processed['label'] = pt_test_df_processed['label'].astype(int)
pt_test_df_processed['label'] = pt_test_df_processed['label'] + 1 # -1->0, 0->1, 1->2
pt_test_df_processed['text'] = pt_test_df_processed['text'].astype(str).fillna('')
pt_test_dataset = Dataset.from_pandas(pt_test_df_processed)
pt_test_encoded = pt_test_dataset.map(tokenize_fn, batched=True)

pt_test_metrics = trainer.evaluate(pt_test_encoded)
print("MahaSent_MR Test metrics:", pt_test_metrics)

Map:   0%|          | 0/2250 [00:00<?, ? examples/s]

MahaSent_MR Test metrics: {'eval_loss': 0.3776489794254303, 'eval_accuracy': 0.86, 'eval_precision_macro': 0.8604049794345436, 'eval_recall_macro': 0.86, 'eval_f1_macro': 0.8592526975330484, 'eval_runtime': 10.5195, 'eval_samples_per_second': 213.889, 'eval_steps_per_second': 13.404, 'epoch': 3.0}


In [ ]:
st_test_df_processed = st_test_df.copy()
st_test_df_processed = st_test_df_processed.rename(columns={"marathi_text": "text"})
st_test_df_processed['label'] = st_test_df_processed['label'].astype(int)
st_test_df_processed['label'] = st_test_df_processed['label'] + 1 # -1->0, 0->1, 1->2
st_test_df_processed['text'] = st_test_df_processed['text'].astype(str).fillna('')
st_test_dataset = Dataset.from_pandas(st_test_df_processed)
st_test_encoded = st_test_dataset.map(tokenize_fn, batched=True)

print("MahaSent_ST Test DataFrame processed and encoded successfully.")
st_test_metrics = trainer.evaluate(st_test_encoded)
print("MahaSent_ST Test metrics:", st_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_ST Test DataFrame processed and encoded successfully.


MahaSent_ST Test metrics: {'eval_loss': 0.5105463862419128, 'eval_accuracy': 0.796, 'eval_precision_macro': 0.7960527521812287, 'eval_recall_macro': 0.7959999999999999, 'eval_f1_macro': 0.7952051972427241, 'eval_runtime': 7.6631, 'eval_samples_per_second': 195.744, 'eval_steps_per_second': 12.267, 'epoch': 3.0}


In [ ]:
uploaded = files.upload()

Saving MahaSent_MR_Test.csv to MahaSent_MR_Test.csv
Saving tweets-test_GT.csv to tweets-test_GT.csv


In [ ]:
mr_test_df = pd.read_csv('MahaSent_MR_Test.csv')
print("MahaSent_PT Test DataFrame loaded successfully.")
print(mr_test_df.head())

mr_test_df_processed = mr_test_df.copy()
mr_test_df_processed = mr_test_df_processed.rename(columns={"marathi_sentence": "text"})
mr_test_df_processed['label'] = mr_test_df_processed['label'].astype(int)
mr_test_df_processed['label'] = mr_test_df_processed['label'] + 1
mr_test_df_processed['text'] = mr_test_df_processed['text'].astype(str).fillna('')
mr_test_dataset = Dataset.from_pandas(mr_test_df_processed)
mr_test_encoded = mr_test_dataset.map(tokenize_fn, batched=True)
print("MahaSent_MR Test DataFrame processed and encoded successfully.")

mr_test_metrics = trainer.evaluate(mr_test_encoded)
print("MahaSent_MR Test metrics:", mr_test_metrics)

MahaSent_PT Test DataFrame loaded successfully.
   Unnamed: 0                                   marathi_sentence  label
0           0  जयदीप त्याच्या आई संहिता हिच्या घटस्फोटाचा भार...     -1
1           1                  मात्र तो त्यात यशस्वी झालेला नाही     -1
2           2  जेव्हा लोकांचा समूह त्यांच्या ड्रॉईंग रूमच्या ...     -1
3           3  दिग्दर्शकाची चित्रपटावरची पकडच पक्की नसल्यानं ...     -1
4           4  तो झिपऱ्याचे जग उत्तम प्रकारे निर्माण करत असता...     -1


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_MR Test DataFrame processed and encoded successfully.


MahaSent_MR Test metrics: {'eval_loss': 0.7456540465354919, 'eval_accuracy': 0.7233333333333334, 'eval_precision_macro': 0.7303914788200624, 'eval_recall_macro': 0.7233333333333333, 'eval_f1_macro': 0.7138276741203787, 'eval_runtime': 4.7642, 'eval_samples_per_second': 314.85, 'eval_steps_per_second': 19.731, 'epoch': 3.0}


In [ ]:
gt_test_df = pd.read_csv('tweets-test_GT.csv')
print("MahaSent_ST Test DataFrame loaded successfully.")
print(gt_test_df.head())

MahaSent_ST Test DataFrame loaded successfully.
                                               tweet  label  political
0                                     कुलकर्णी हे बघ     -1         -1
1  हे प्रत्येक पक्ष्याच्या सतरंज्या उचल्यागीरी कर...     -1         -1
2  जी च्या दृष्टीने पुणे जिल्ह्यात एकही लायक आमदा...     -1          1
3  शेणकयांच वजन    किलो त्याच्यापेक्षा वजनदार भां...     -1          0
4  पार होती नव्हती इज्जत काढली....सरकारी लोक स्वत...     -1          1


In [ ]:
gt_test_df_processed = gt_test_df.copy()
gt_test_df_processed = gt_test_df_processed.rename(columns={"tweet": "text"})
gt_test_df_processed['label'] = gt_test_df_processed['label'].astype(int)
gt_test_df_processed['label'] = gt_test_df_processed['label'] + 1
gt_test_df_processed['text'] = gt_test_df_processed['text'].astype(str).fillna('')
gt_test_dataset = Dataset.from_pandas(gt_test_df_processed)
gt_test_encoded = gt_test_dataset.map(tokenize_fn, batched=True)

gt_test_metrics = trainer.evaluate(gt_test_encoded)
print("MahaSent_GT Test metrics:", gt_test_metrics)


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_GT Test metrics: {'eval_loss': 0.66032475233078, 'eval_accuracy': 0.7453333333333333, 'eval_precision_macro': 0.7539723592856548, 'eval_recall_macro': 0.7453333333333333, 'eval_f1_macro': 0.7444419117675257, 'eval_runtime': 6.6247, 'eval_samples_per_second': 226.424, 'eval_steps_per_second': 14.189, 'epoch': 3.0}
